In [1]:
import sys
import os
sys.path.append('../metal')
sys.path.append('../heart-MRI-pytorch')
sys.path.append('../data')
sys.path.append('../../sequential_ws')

In [2]:
import numpy as np
import argparse
import torch
import logging
import warnings
import pandas
from glob import glob
from scipy.sparse import csr_matrix
import torchvision
import torch.nn as nn
from torch.nn.functional import normalize
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from utils import *
from metal.metrics import metric_score
from metal.label_model import LabelModel
from metal.label_model.baselines import MajorityLabelVoter
from metal.analysis import lf_summary, confusion_matrix
from DP.label_model import DPLabelModel, optimize

In [3]:
logger = logging.getLogger(__name__)
warnings.filterwarnings("ignore", category=DeprecationWarning)
#warnings.filterwarnings("ignore", category=UndefinedMetricWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

## Reading Data

In [4]:
def read_labels(label_list):
    '''
    Function to read labels given list of labels
    Returns a sparse matrix for LFs
    Returns numpy array for true labels
    
    Input
    ----
    label_list: list of labels
    
    Output
    -----
    L: sparse matrix (#patients*#frames, #LFs)
    or L: numpy array (#patients*#frames,)
    '''
    
    L = []
    for index in range(len(label_list)):
        L.append(np.load(label_list[index]))

    L = np.squeeze(np.array(L))
    
    # reshaping array from (PID,frames,) -> (PID*frames,)
    m = L.shape[0]
    n = L.shape[1]
    if(len(L.shape) == 2): # true labels 
        L = np.reshape(L,(m*n,))
        L = L+1 # changing from 0-indexing to 1-indexing
    else:
        L = csr_matrix(np.reshape(L,(m*n,L.shape[2])))

    return L

In [5]:
def load_labels(args):
    '''
    Script to read labels using input args
    '''
    L = {}
    Y = {}

    #train_lf_list = glob(args.train + '/lf_labels/*.npy') 
    L["train"] = read_labels(glob(args["train"] + '/lf_labels/*.npy'))
    L["dev"] = read_labels(glob(args["dev"] + '/lf_labels/*.npy'))
    L["test"] = read_labels(glob(args["test"] + '/lf_labels/*.npy'))

    #import ipdb; ipdb.set_trace()
    Y["dev"] = read_labels(glob(args["dev"] + '/true_labels/*.npy'))
    Y["test"] = read_labels(glob(args["test"] + '/true_labels/*.npy'))	

    return L,Y

In [6]:
# loading data
args = {}
args["train"] = '../data/open_close/train'
args["dev"] = '../data/open_close/dev'
args["test"] = '../data/open_close/test'

L,Y = load_labels(args) 

In [7]:
#print(L["train"].todense().shape) # (18850,5)
#print(L["dev"].todense().shape) # (1500,5)
#print(Y["dev"].shape) # (1500,)

In [8]:
# labelling functions analysis
print(lf_summary(L["dev"], Y = Y["dev"]))

     Polarity  Coverage  Overlaps  Conflicts  Correct  Incorrect  Emp. Acc.
0  [1.0, 2.0]       1.0       1.0   0.212667     1359        141   0.906000
1  [1.0, 2.0]       1.0       1.0   0.212667     1312        188   0.874667
2  [1.0, 2.0]       1.0       1.0   0.212667     1361        139   0.907333
3  [1.0, 2.0]       1.0       1.0   0.212667     1309        191   0.872667
4  [1.0, 2.0]       1.0       1.0   0.212667     1377        123   0.918000


### Majority Vote

In [9]:
# majority vote of LFs
mv = MajorityLabelVoter(seed=123)
print('Majority Label Voter Metrics:')
mv_score = mv.score((L["dev"], Y["dev"]), metric=['accuracy','precision', 'recall', 'f1'])

Majority Label Voter Metrics:
Accuracy: 0.915
Precision: 0.894
Recall: 0.920
F1: 0.907
        y=1    y=2   
 l=1    625    74    
 l=2    54     747   


### Metal Label Model

In [10]:
# defining parameters
num_classes = 2
if (torch.cuda.is_available()):
    device = 'cuda:0'
else:
    device = 'cpu'
    
print(device)

cuda:0


In [11]:
# training label model - no temporal modelling
label_model = LabelModel(k=num_classes, seed=123)
label_model.train_model(L["train"], Y["dev"], n_epochs = 500, log_train_every = 50)

# evaluating label model
print('Trained Label Model Metrics:')
lm_score = label_model.score((L["dev"], Y["dev"]), metric=['accuracy','precision', 'recall', 'f1'])

Computing O...
Estimating \mu...
[50 epo]: TRAIN:[loss=0.017]
[100 epo]: TRAIN:[loss=0.002]
[150 epo]: TRAIN:[loss=0.002]
[200 epo]: TRAIN:[loss=0.002]
[250 epo]: TRAIN:[loss=0.002]
[300 epo]: TRAIN:[loss=0.002]
[350 epo]: TRAIN:[loss=0.002]
[400 epo]: TRAIN:[loss=0.002]
[450 epo]: TRAIN:[loss=0.002]
[500 epo]: TRAIN:[loss=0.002]
Finished Training
Trained Label Model Metrics:
Accuracy: 0.913
Precision: 0.885
Recall: 0.928
F1: 0.906
        y=1    y=2   
 l=1    630    82    
 l=2    49     739   


## DP Model 

In [16]:
# training label model without temporal modelling
# ( this should reproduce the results above )
m_per_task = L["train"].todense().shape[1] # 5
MRI_data_naive = {'Li_train': torch.LongTensor(np.array(L["train"].todense().astype(int))),
                'Li_dev': torch.LongTensor(np.array(L["dev"].todense().astype(int))),
                'R_dev':Y["dev"]}

MRI_data_naive['class_balance'] = torch.FloatTensor([0.5,0.5]).to(device)

In [13]:
# training naive model 
naive_model = DPLabelModel(m=m_per_task, 
                       T=1,
                       edges=[],
                       coverage_sets=[[0,]]*m_per_task,
                       mu_sharing=[[i,] for i in range(m_per_task)],
                       phi_sharing=[],
                       device=device,
                       class_balance=MRI_data_naive['class_balance'], 
                       seed=0)

optimize(naive_model, L_hat=MRI_data_naive['Li_train'], num_iter=3000, lr=1e-3, momentum=0.8, clamp=True, seed=0)

iteration=0 loss=55.706329345703125
iteration=300 loss=0.08146391808986664
iteration=600 loss=0.0741795152425766
iteration=900 loss=0.06845731288194656
iteration=1200 loss=0.06415102630853653
iteration=1500 loss=0.06096632033586502
iteration=1800 loss=0.05861331522464752
iteration=2100 loss=0.05686088278889656
iteration=2400 loss=0.05554001033306122
iteration=2700 loss=0.05453081429004669
iteration=2999 loss=0.05400734022259712


In [18]:
# evaluating naive model 
naive_model = naive_model.cuda()
R_pred = naive_model.predict( MRI_data_naive['Li_dev'] ).data.numpy()
R_pred = 2 - R_pred
#print(R_pred)
#print(MRI_data_naive['R_dev'])

for metric in ['accuracy', 'f1', 'recall', 'precision']:
    score = metric_score(MRI_data_naive['R_dev'], R_pred, metric)
    print(f"{metric.capitalize()}: {score:.3f}")
    
    

RuntimeError: Expected object of backend CPU but got backend CUDA for argument #2 'mat2'

### Temporal Modelling

In [33]:
# training label model with temporal modelling
# reshaping dataset
num_frames = 50
n_patients_train = round(L["train"].todense().shape[0]/num_frames) #(377)
n_patients_dev = round(L["dev"].todense().shape[0]/num_frames) #(30)
Ltrain = np.reshape(np.array(L["train"].todense()),(n_patients_train,num_frames,-1))
Ldev = np.reshape(np.array(L["dev"].todense()),(n_patients_dev,num_frames,-1))
Ydev = np.reshape(Y["dev"],(n_patients_dev,num_frames))
# print(Ltrain.shape) # (377,50,5)
#print(Ldev.shape) # (30,50,5)
#print(Ydev.shape) # (30,50)

In [45]:
# subsampling
# selecting frames 3,13,23,33,43
indices = np.linspace(2,42,5).astype(int)
m_per_task = 5
T = 5

Ltrain_small = Ltrain[:,indices,:] # shape (377,5,5)
Ldev_small = Ldev[:,indices,:] # shape (30,5,5)
Ydev_small = Ydev[:,indices] # shape (30,5)

Ltrain_small = np.reshape(Ltrain_small,((n_patients_train*T),m_per_task)) # shape (1885,5)
Ldev_small = np.reshape(Ldev_small,((n_patients_dev*T),m_per_task)) # shape (150,5)
Ydev_small = np.reshape(Ydev_small,((n_patients_dev*T),)) # shape (150,)


In [46]:

MRI_data_temporal = {'Li_train': torch.LongTensor(Ltrain_small).view(n_patients_train, (m_per_task*T)), 
                    'Li_dev': torch.LongTensor(Ldev_small).view(n_patients_dev, (m_per_task*T)), 
                    'R_dev':torch.LongTensor(Ydev_small)[::T]*(2**T-1),
                    'm': m_per_task*T,
                    'T': T }

MRI_data_temporal['class_balance'] = normalize((MRI_data_temporal['R_dev'].unsqueeze(1)==torch.arange(2**T, device=device).unsqueeze(0)).sum(0).float(), 
                                                dim=0, p=1)

In [ ]:
max_seed = 10
temporal_models = [None,]*max_seed
for seed in range(max_seed):
    markov_model = DPLabelModel(m=m_per_task*T, 
                                T=T,
                                edges=[(i,i+m_per_task) for i in range((T-1)*m_per_task)],
                                coverage_sets=[[t,] for t in range(T) for _ in range(m_per_task)],
                                mu_sharing=[[t*m_per_task+i for t in range(T)] for i in range(m_per_task)],
                                phi_sharing=[[(t*m_per_task+i, (t+1)*m_per_task+i) for t in range(T-1)] for i in range(m_per_task)],
                                device=device,
                                class_balance=MRI_data_temporal['class_balance'],
                                seed=seed)
    optimize(markov_model, L_hat=MRI_data_temporal['Li_train'], num_iter=1000, lr=1e-5, momentum=0.8, clamp=True, 
             verbose=False, seed=seed)
    temporal_models[seed] = markov_model

In [ ]:
for seed, model in enumerate(temporal_models):
    R_pred = model.predict(MRI_data_temporal['Li_dev'])
    F1 = metric_score(MRI_data_temporal['R_dev'].cpu()>0, R_pred.cpu()>0, 'f1')
    accuracy = metric_score(MRI_data_temporal['R_dev'].cpu(), R_pred.cpu(), 'accuracy')
    print(f"seed={seed}  accuracy={accuracy:.3f}  F1={F1:.3f}")